<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2019/images/lec_title.png?raw=true" alt="2019年度ゲノム情報解析入門" height="100px" align="middle">

# 機械学習 - ゲノム解析 -


　ゲノム解析においても、機械学習はよく利用されています。

例えば、
- DNA配列やアミノ酸配列からパターンを読み取り、機能や構造、種類などを予測する（機能予測、構造予測、トポロジー予測など）
- ゲノム全体にわたる遺伝的変異から作物の収穫量を予測するモデルを作成し、育種に活用する（Genomic selection）

## アミノ酸配列から膜貫通タンパク質を予測する

　遺伝子産物であるタンパク質は、アミノ酸で構成されています。タンパク質の種類や機能を知ることは、遺伝子（タンパク質）の働きを理解するための第一歩になります。

　例えば、下記に代表されるように、アミノ酸配列から[膜タンパク質（Membrane protein）](https://en.wikipedia.org/wiki/Membrane_protein)かどうかを予測するツールがあります。

- [TMHMM](http://www.cbs.dtu.dk/services/TMHMM/)
- [SOSUI](http://harrier.nagahama-i-bio.ac.jp/sosui/sosui_submit.html)

　ここでは、そのようなツールの簡易版を作成していきます。

<small>*※ 上述のTMHMMやSOSUIは、与えられたアミノ酸配列が膜タンパク質かどうかの予測だけでなく、膜貫通領域の部位がどこかも予測できます。今回作成する簡易版ツールは、「膜タンパク質の予測」のみをおこなうツールです。*</small>

### 使用するデータセット

　アミノ酸配列のようなデータセットを用いる場合、そのままでは機械学習のデータとして利用できません。まず、数値化をおこなう必要があります。

- [数値化前のデータセット](https://github.com/CropEvol/lecture/blob/master/textbook_2019/dataset/membrane_dataset.csv)

　数値化の一例として、ここでは「アミノ酸配列」を「各アミノ酸の割合」に変換しています。以降の解析では、配列中に含まれる各アミノ酸の割合を、解析データとして使っていきます。  
<small>※ このようなゲノム情報の数値化も「前処理」に含まれます。</small>

- [数値化後のデータセット](https://github.com/CropEvol/lecture/blob/master/textbook_2019/dataset/membrane_dataset.AAfreq.csv)
  - `label`: 目的変数（正解ラベル）
  - `A`-`Y`: 説明変数（20個のアミノ酸の割合）。各アミノ酸は一文字表記にしています（詳しくは[こちら](https://ja.wikipedia.org/wiki/%E3%82%BF%E3%83%B3%E3%83%91%E3%82%AF%E8%B3%AA%E3%82%92%E6%A7%8B%E6%88%90%E3%81%99%E3%82%8B%E3%82%A2%E3%83%9F%E3%83%8E%E9%85%B8)）。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2019/images/AA_to_dataset.png?raw=true" alt="AminoAcids_to_dataset" height="150px">

In [0]:
# データセットのダウンロード
!wget -O membrane_dataset.AAfreq.csv https://raw.githubusercontent.com/CropEvol/lecture/master/textbook_2019/dataset/membrane_dataset.AAfreq.csv

### 前処理

　予測モデルを作成する前に、前処理をおこなう必要があります。ここでは、次の3つの前処理をおこなっています。

1. データの分割
  - トレーニング用のデータとテスト用のデータを用意します。
  - ここでは、トレーニングデータ70%、テストデータ30%に分割。

1. スケーリング
  - 各説明変数の尺度を揃えます。
  - ここでは、各説明変数を正規化（0~1の範囲のデータに変換）。  
  <small> ※ 「アミノ酸の割合」をデータとして扱うため、今回この操作はあまり重要ではありません。</small>

1. ラベルの数値変換
  - ラベル（membrane, non_membrane）を数値データ（0,1）に変換します。


　次のセルを実行して、前処理をおこなってください。

　


In [0]:
import numpy as np
import pandas as pd

# 読み込み
df = pd.read_csv("membrane_dataset.AAfreq.csv", sep=',', header=0)
x = np.array(df.iloc[:, 1:])  # 説明変数（特徴量）
y = np.array(df["label"])   # 目的変数（ラベル）

# 前処理
# (1) データの分割
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, stratify=y, random_state=0)

# (2) スケーリング
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
x_train_sc = mms.fit_transform(x_train) # トレーニングデータ
x_test_sc = mms.transform(x_test) # テストデータ

# (3) ラベルの数値変換
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train_le = le.fit_transform(y_train) # トレーニングデータ
y_test_le = le.transform(y_test) # テストデータ

### モデルの作成・学習・評価

　モデルの作成、学習、評価をおこなっていきます。scikit-learnを使うと、モデルの作成と学習は、次の3行でおこなえます。

```python
# モデルの作成と学習
from sklearn.機能 import 関数 
モデル変数 = 関数(オプション)
モデル変数.fit(x, y)
```

　また、モデルの評価（正解率の算出）も1行でおこなえます。
```python
# モデルの評価
モデル変数.score(x, y)
```

In [0]:
# 使用するデータ
x_train, y_train = x_train_sc, y_train_le  # トレーニングデータ
x_test, y_test  = x_test_sc, y_test_le   # テストデータ

# モデルを作成
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(solver='lbfgs', random_state=0)

# モデルをトレーニング
clf.fit(x_train, y_train)

# 評価（正解率）
print("training data: ", clf.score(x_train, y_train)) # トレーニングデータ
print("test data: ",    clf.score(x_test, y_test))  # テストデータ

### 予測

　上で作成したモデルを使って、新しいデータのラベル（membrane/non_membrane）を予測します。

予測したい新しいデータ（4つのタンパク質）
- [数値化前](https://github.com/CropEvol/lecture/blob/master/textbook_2019/dataset/membrane_newdata.csv)
- [数値化後](https://github.com/CropEvol/lecture/blob/master/textbook_2019/dataset/membrane_newdata.AAfreq.csv)
  - SUT1_ORYSJ: [Sucrose transport protein SUT1 / Oryza sativa](https://www.uniprot.org/uniprot/Q10R54)
  - SCAM1_ORYSJ: [Secretory carrier-associated membrane protein 1 / Oryza sativa](https://www.uniprot.org/uniprot/Q8H5X5)
  - MADS7_ORYSJ: [MADS-box transcription factor 7 / Oryza sativa](https://www.uniprot.org/uniprot/Q0J466)
  - GAOX2_ORYSJ: [Gibberellin 20 oxidase 2](https://www.uniprot.org/uniprot/Q0JH50)

In [0]:
# 新しいデータのダウンロード
!wget -O membrane_newdata.AAfreq.csv https://raw.githubusercontent.com/CropEvol/lecture/master/textbook_2019/dataset/membrane_newdata.AAfreq.csv

In [0]:
# 読み込み
new_df = pd.read_csv("membrane_newdata.AAfreq.csv", sep=',', header=0)
x_new = np.array(new_df.iloc[:, 1:])

# 前処理（正規化）
x_new = mms.transform(x_new)

# 予測
y_pred = clf.predict(x_new)
print(le.inverse_transform(y_pred))

### 実習

　モデルの作成・学習の部分のコードを埋めて、下記の目標をクリアした予測モデルを構築してください。

- 目標: テストデータの正解率 91%以上


　使用する分類手法は自由に選んでください（下記にリンクをクリックすると、scikit-learnのマニュアルページが開きます）。
  - [ロジスティック回帰](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)
  - [サポートベクトルマシン](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)
  - [決定木](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)
  - [ランダムフォレスト](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)
  - [ニューラルネットワーク](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html)

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier


# ===== 編集エリア: start ===== 
# モデルを作成
clf = 

# ===== 編集エリア: end  ===== 

# モデルをトレーニング
clf.fit(x_train, y_train)

# 評価（正解率）
print("training data: ", clf.score(x_train, y_train)) # トレーニングデータ
print("test data: ",    clf.score(x_test, y_test))  # テストデータ

# 予測
y_pred = clf.predict(x_new)
print(le.inverse_transform(y_pred))

#### 解答例

　下記の分類手法のいずれかで、目標を達成したモデルを構築できます。
- サポートベクトルマシン
- ランダムフォレスト
- ニューラルネットワーク

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier

# ===== 編集エリア: start ===== 
# モデルを作成
clf = SVC(kernel="rbf", random_state=0)
#clf = RandomForestClassifier(n_estimators=100, max_depth=6, random_state=0)
#clf = MLPClassifier(hidden_layer_sizes=(12,12,12,), max_iter=5000, random_state=0)

# ===== 編集エリア: end  ===== 

# モデルをトレーニング
clf.fit(x_train, y_train)

# 評価（正解率）
print("training data: ", clf.score(x_train, y_train)) # トレーニングデータ
print("test data: ",    clf.score(x_test, y_test))  # テストデータ

# 予測
y_pred = clf.predict(x_new)
print(le.inverse_transform(y_pred))

### 良いモデルを構築するために重要なこと

　上述の実習では、どの分類手法を使うかや、パラメータの値をどうするか、に焦点を当てていましたが、機械学習で一番重要な部分は「前処理: データセットの準備」です。

　今回の場合、アミノ酸配列の数値化方法をもう少し工夫をすることで、予測モデルを改善できるかもしれません。一般的に、膜タンパク質には、15-30個の連続した疎水性アミノ酸領域を配列中に保有しています。これは、細胞膜を貫通する領域（膜貫通領域）が、疎水性のアミノ酸で構成されているためです。したがって、このような情報を考慮したデータセットを準備することで、予測モデルの改善が期待できます。

<small>※ 例えば、実習では「個々のアミノ酸の割合」をデータに使いましたが、「連続する複数個のアミノ酸の割合（アミノ酸配列の割合）」を使用することで、予測モデルを改善できるかもしれません。</small>

　**良いデータセットがなければ、良いモデルはできません。**

---
## まとめ

　次世代シーケンシング技術の発達により、ゲノムワイドなデータを容易に取得できるようになってきました。アクセス可能なゲノムデータの増加に伴い、機械学習を利用したゲノムデータ解析が"あたり前"の時代になってきています。

　ゲノムデータを解析するための重要なポイントは、ゲノムデータを数値化することです。しかし、闇雲に数値化すれば良いわけではありません。「良いモデル」を構築するためには、適切に情報を数値化して、「良いデータセット」を作る必要があります。  
　データセットができれば、ここ数回の実習で見てきたように、自分で機械学習ベースのデータ解析をおこなうことが可能です。

　今後、自身の研究や仕事でデータ解析をおこなうことがあるかと思います。その際には、自分でコーディングして解析することにチャレンジしてみてください。